# Solution to a queue problem using TD Learning

# 1 - Imports

In [52]:
%load_ext autoreload
%autoreload 2
import class_queue as cq
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2 - Env creation

## 2.1 - Creation an print of the env properties

In [53]:
queue = cq.Queue(num_sinks=6, array_utilities=['soap', 'soap', 'towel', 'soap', 'towel', 'none'])

In [54]:
print(f'Sink count: {queue.num_sinks} sinks')
print(f'Utilities: {queue.array_utilities}')
print(f'Sink avaiability: {queue.sinks_availability}\n')

print(f'Queue growth: {queue.queue_growth} per iterarion')
print(f'Max away: {queue.away_max_size}')
print(f'Num agents: {len(queue.agents)}')
print(f'Queue occupation status: {queue.get_occupation()}\n')

print(f'Time spent for each task (mean and sd): {queue.queue_times}')
print(f'Agents in queue: {queue.agents}')

Sink count: 6 sinks
Utilities: ['soap', 'soap', 'towel', 'soap', 'towel', 'none']
Sink avaiability: 000000

Queue growth: 0.5 per iterarion
Max away: 5
Num agents: 0
Queue occupation status: LOW

Time spent for each task (mean and sd): {'soap': [10, 1], 'towel': [5, 0.5], 'washing': [3, 0.5]}
Agents in queue: []


## 2.2 - Randomization of the env state

In [55]:
queue.reset_state()
queue.randomize_state()
print(f'Num agents generated: {len(queue.agents)}')
print(f'Agents postions, need and iterations_until_action: \n{[(a.position, a.need, a.iterations_until_action) for a in queue.agents]}')

Num agents generated: 10
Agents postions, need and iterations_until_action: 
[('3', 'soap', 0), ('away', 'soap', 0), ('1', 'towel', 0), ('4', 'wait', 0), ('2', 'wait', 0), ('away', 'washing', 0), ('5', 'washing', 0), ('6', 'wait', 0), ('away', 'soap', 0), ('away', 'towel', 0)]


In [86]:
queue.get_agents_ready_for_action()
queue.sample_action_time('soap')

8

In [57]:
agent = queue.agents[0]
list(queue.q_table.iloc[0].values[:-1]), [agent.position, agent.need, queue.sinks_availability]

(['1', 'soap', '000000', 'LOW'], ['3', 'soap', '000000'])

In [139]:
np.random.choice(1)+1

1